<a href="https://colab.research.google.com/github/Mr-Kondo/Open/blob/main/8_3_simcse_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第8章 文埋め込み

## 8.3 文埋め込みモデルの実装

### 8.3.1 教師なしSimCSEの実装

#### 準備

In [ ]:
# 必要なパッケージをインストールする
!pip install datasets scipy transformers[ja,torch]

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定する
set_seed(42)

#### データセットの読み込みと前処理

In [ ]:
pip install fsspec==2023.9.2

In [ ]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/jawiki-sentencesのリポジトリから
# Wikipediaの文のデータを読み込み、SimCSEの訓練セットとして使用する
unsup_train_dataset = load_dataset(
    "llm-book/jawiki-sentences", split="train"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
# 訓練セットの形式と事例数を確認する
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 24387500
})


In [ ]:
# 訓練セットの内容を確認する
for i, text in enumerate(unsup_train_dataset[:50]["text"]):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

In [ ]:
# 訓練セットから空行の事例を除外する
unsup_train_dataset = unsup_train_dataset.filter(
    lambda example: example["text"].strip() != ""
)

Filter:   0%|          | 0/24387500 [00:00<?, ? examples/s]

In [ ]:
# 訓練セットをシャッフルし、最初の100万事例を取り出す
unsup_train_dataset = unsup_train_dataset.shuffle().select(
    range(1000000)
)
# パフォーマンスの低下を防ぐため、シャッフルされた状態の訓練セットを
# ディスクに書き込む
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [ ]:
# 前処理後の訓練セットの形式と事例数を確認する
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


In [ ]:
# 前処理後の訓練セットの内容を確認する
for i, text in enumerate(unsup_train_dataset[:10]["text"]):
    print(i, text)

0 2005年の時点で、10,000人ものウズベキスタン人が韓国での労働に従事しており、その大部分が高麗人である。
1 小学5年生(11歳)の時から芸能活動を開始。
2 i ħ d d t | ψ ( t ) ⟩ = L ^ | ψ ( t ) ⟩ {\displaystyle i\hbar {\frac {d}{dt}}|\psi (t)\rangle ={\hat {L}}|\psi (t)\rangle }
3 安土宗論(あづちしゅうろん)は、1579年(天正7年)、安土城下の浄厳院で行われた浄土宗と法華宗の宗論。
4 1927年 オーストラリア選手権(1927ねんオーストラリアせんしゅけん、1927 Australian Championships)に関する記事。
5 さらにマップ上で最大8つまでしか建築できず(司令官アビリティの”解体”か設置したプレイヤー自らが出向いて解体する必要がある)
6 特に誉淳が1827年から作成した『古瓦譜』は畿内で600点以上の拓本を蒐集し、瓦当文様に着目したうえで編年を試みている。
7 マルクス主義者を広言し、メキシコ共産党の敵であり味方であった。
8 ICHILLIN'(アイチリン、朝: 아이칠린)は、韓国の7人組女性アイドルグループ。
9 マークVIは1983年にモデルサイクルを終了し、1984年のマークVII(英語版)はフルサイズセグメントから撤退し、マークシリーズは異なるセグメントに移行した。


In [ ]:
# Hugging Face Hubのllm-book/JGLUEのリポジトリから
# JSTSデータセットの訓練セットと検証セットを読み込み、
# それぞれをSimCSEの検証セットとテストセットとして使用する
valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="train"
)
test_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="validation"
)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

#### トークナイザと collate 関数の準備

In [ ]:
from transformers import AutoTokenizer

# Hugging Face Hubにおけるモデル名を指定する
base_model_name = "cl-tohoku/bert-base-japanese-v3"
# モデル名からトークナイザを初期化する
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

In [ ]:
import torch
from torch import Tensor
from transformers import BatchEncoding

def unsup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """教師なしSimCSEの訓練セットのミニバッチを作成"""
    # ミニバッチに含まれる文にトークナイザを適用する
    tokenized_texts = tokenizer(
        [example["text"] for example in examples],
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文）に対してi列目の事例（文）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    return {
        "tokenized_texts_1": tokenized_texts,
        "tokenized_texts_2": tokenized_texts,
        "labels": labels,
    }

In [ ]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """SimCSEの検証・テストセットのミニバッチを作成"""
    # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれに
    # トークナイザを適用する
    tokenized_texts_1 = tokenizer(
        [example["sentence1"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized_texts_2 = tokenizer(
        [example["sentence2"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文1）に対して
    # i列目の事例（文2）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    # データセットに付与された類似度スコアのTensorを作成する
    label_scores = torch.tensor(
        [example["label"] for example in examples]
    )

    return {
        "tokenized_texts_1": tokenized_texts_1,
        "tokenized_texts_2": tokenized_texts_2,
        "labels": labels,
        "label_scores": label_scores,
    }

#### モデルの準備

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSEModel(nn.Module):
    """SimCSEのモデル"""

    def __init__(
        self,
        base_model_name: str,
        mlp_only_train: bool = False,
        temperature: float = 0.05,
    ):
        """モデルの初期化"""
        super().__init__()

        # モデル名からエンコーダを初期化する
        self.encoder = AutoModel.from_pretrained(base_model_name)
        # パラメータをメモリ上に隣接した形で配置
        # これを実行しない場合、モデルの保存でエラーになることがある
        for param in self.encoder.parameters():
            param.data = param.data.contiguous()
        # MLP層の次元数
        self.hidden_size = self.encoder.config.hidden_size
        # MLP層の線形層
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        # MLP層の活性化関数
        self.activation = nn.Tanh()

        # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
        self.mlp_only_train = mlp_only_train
        # 交差エントロピー損失の計算時に使用する温度
        self.temperature = temperature

    def encode_texts(self, tokenized_texts: BatchEncoding) -> Tensor:
        """エンコーダを用いて文をベクトルに変換"""
        # トークナイズされた文をエンコーダに入力する
        encoded_texts = self.encoder(**tokenized_texts)
        # モデルの最終層の出力（last_hidden_state）の
        # [CLS]トークン（0番目の位置のトークン）のベクトルを取り出す
        encoded_texts = encoded_texts.last_hidden_state[:, 0]

        # self.mlp_only_trainのフラグがTrueに設定されていて
        # かつ訓練時でない場合、MLP層の変換を適用せずにベクトルを返す
        if self.mlp_only_train and not self.training:
            return encoded_texts

        # MLP層によるベクトルの変換を行う
        encoded_texts = self.dense(encoded_texts)
        encoded_texts = self.activation(encoded_texts)

        return encoded_texts

    def forward(
        self,
        tokenized_texts_1: BatchEncoding,
        tokenized_texts_2: BatchEncoding,
        labels: Tensor,
        label_scores: Tensor | None = None,
    ) -> ModelOutput:
        """モデルの前向き計算を定義"""
        # 文ペアをベクトルに変換する
        encoded_texts_1 = self.encode_texts(tokenized_texts_1)
        encoded_texts_2 = self.encode_texts(tokenized_texts_2)

        # 文ペアの類似度行列を作成する
        sim_matrix = F.cosine_similarity(
            encoded_texts_1.unsqueeze(1),
            encoded_texts_2.unsqueeze(0),
            dim=2,
        )

        # 交差エントロピー損失を求める
        loss = F.cross_entropy(sim_matrix / self.temperature, labels)

        # 性能評価に使用するため、正例ペアに対するスコアを類似度行列から取り出す
        positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
        positive_scores = torch.masked_select(
            sim_matrix, positive_mask
        )

        return ModelOutput(loss=loss, scores=positive_scores)

# 教師なしSimCSEのモデルを初期化する
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

#### `Trainer` の準備

In [ ]:
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
    """
    モデルが予測したスコアと評価用データのスコアの
    スピアマンの順位相関係数を計算
    """
    scores = p.predictions
    labels, label_scores = p.label_ids

    spearman = spearmanr(scores, label_scores).statistic

    return {"spearman": spearman}

In [ ]:
from transformers import TrainingArguments

# 教師なしSimCSEの訓練のハイパーパラメータを設定する
unsup_training_args = TrainingArguments(
    output_dir="outputs_unsup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=64,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=64,  # 評価時のバッチサイズ
    learning_rate=3e-5,  # 学習率
    num_train_epochs=1,  # 訓練エポック数
    eval_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
    report_to="none",  # 外部ツールへのログを無効化
)

In [ ]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import Trainer

class SimCSETrainer(Trainer):
    """SimCSEの訓練に使用するTrainer"""

    def get_eval_dataloader(
        self, eval_dataset: Dataset | None = None
    ) -> DataLoader:
        """
        検証・テストセットのDataLoaderでeval_collate_fnを使うように
        Trainerのget_eval_dataloaderをオーバーライド
        """
        if eval_dataset is None:
            eval_dataset = self.eval_dataset

        return DataLoader(
            eval_dataset,
            batch_size=64,
            collate_fn=eval_collate_fn,
            pin_memory=True,
        )

# 教師なしSimCSEのTrainerを初期化する
unsup_trainer = SimCSETrainer(
    model=unsup_model,
    args=unsup_training_args,
    data_collator=unsup_train_collate_fn,
    train_dataset=unsup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

#### 訓練の実行

In [ ]:
# 教師なしSimCSEの訓練を行う
unsup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,0.001800,2.616623,0.697291
500,0.000200,2.486944,0.719534
750,0.000200,2.446804,0.727137
1000,0.000300,2.401694,0.732074
1250,0.000500,2.367676,0.741027
1500,0.000100,2.395255,0.745788
1750,0.000100,2.410435,0.744032
2000,0.000200,2.403177,0.750924
2250,0.000100,2.396734,0.746827
2500,0.000200,2.377998,0.749588


TrainOutput(global_step=15625, training_loss=0.0001299928292632103, metrics={'train_runtime': 5694.729, 'train_samples_per_second': 175.601, 'train_steps_per_second': 2.744, 'total_flos': 0.0, 'train_loss': 0.0001299928292632103, 'epoch': 1.0})

#### 性能評価

In [ ]:
# 検証セットで教師なしSimCSEのモデルの評価を行う
unsup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.2657642364501953,
 'eval_spearman': 0.7594557089091337,
 'eval_runtime': 14.6586,
 'eval_samples_per_second': 849.401,
 'eval_steps_per_second': 13.303,
 'epoch': 1.0}

In [ ]:
# テストセットで教師なしSimCSEのモデルの評価を行う
unsup_trainer.evaluate(test_dataset)

{'eval_loss': 2.1492629051208496,
 'eval_spearman': 0.7876485055178543,
 'eval_runtime': 1.7853,
 'eval_samples_per_second': 816.101,
 'eval_steps_per_second': 12.883,
 'epoch': 1.0}

#### トークナイザとモデルの保存

In [ ]:
# 教師なしSimCSEのエンコーダを保存
encoder_path = "outputs_unsup_simcse/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

('outputs_unsup_simcse/encoder/tokenizer_config.json',
 'outputs_unsup_simcse/encoder/special_tokens_map.json',
 'outputs_unsup_simcse/encoder/vocab.txt',
 'outputs_unsup_simcse/encoder/added_tokens.json')

#### Google ドライブへの保存

In [ ]:
from google.colab import drive

# Googleドライブをマウントする
drive.mount("drive")

Mounted at drive


In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!mkdir -p drive/MyDrive/llm-book
!cp -r outputs_unsup_simcse drive/MyDrive/llm-book

### 8.3.2 教師あり SimCSE の実装

#### 準備

In [ ]:
# 乱数のシードを設定する
set_seed(42)

#### データセットの読み込みと前処理

In [ ]:
# Hugging Face Hubのllm-book/jsnliのリポジトリから
# JSNLIの訓練セットを読み込む
jsnli_dataset = load_dataset("llm-book/jsnli", split="train")

Generating train split:   0%|          | 0/533005 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3916 [00:00<?, ? examples/s]

In [ ]:
# JSNLIの訓練セットの形式と事例数を確認する
print(jsnli_dataset)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 533005
})


In [ ]:
from pprint import pprint

# JSNLIの訓練セットの内容を確認する
pprint(jsnli_dataset[0])
pprint(jsnli_dataset[1])

{'hypothesis': '男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。',
 'label': 'neutral',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'hypothesis': '女性 が 畑 で 踊って い ます 。',
 'label': 'contradiction',
 'premise': '茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。'}


In [ ]:
import csv
import random
from typing import Iterator

# JSNLIの訓練セットから、前提文とラベルごとに仮説文をまとめたdictを作成する
premise2hypotheses = {}

premises = jsnli_dataset["premise"]  # 前提文
hypotheses = jsnli_dataset["hypothesis"]  # 仮説文
labels = jsnli_dataset["label"]  # ラベル

for premise, hypothesis, label in zip(premises, hypotheses, labels):
    if premise not in premise2hypotheses:
        premise2hypotheses[premise] = {
            "entailment": [],
            "neutral": [],
            "contradiction": [],
        }

    premise2hypotheses[premise][label].append(hypothesis)

In [ ]:
def generate_sup_train_example() -> Iterator[dict[str, str]]:
    """教師ありSimCSEの訓練セットの事例を生成"""
    # JSNLIのデータから (前提文,「含意」ラベルの仮説文,「矛盾」ラベルの仮説文)
    # の三つ組を生成する
    for premise, hypotheses in premise2hypotheses.items():
        # 「矛盾」ラベルの仮説文が一つもない事例はスキップする
        if len(hypotheses["contradiction"]) == 0:
            continue

        # 「含意」ラベルの仮説文一つにつき、「矛盾」ラベルの仮説文一つを
        # ランダムに関連付ける
        for entailment_hypothesis in hypotheses["entailment"]:
            contradiction_hypothesis = random.choice(
                hypotheses["contradiction"]
            )
            # (前提文,「含意」ラベルの仮説文,「矛盾」ラベルの仮説文) の三つ組を
            # dictとして生成する
            yield {
                "premise": premise,
                "entailment_hypothesis": entailment_hypothesis,
                "contradiction_hypothesis": contradiction_hypothesis,
            }

# 定義したジェネレータ関数を用いて、教師ありSimCSEの訓練セットを構築する
sup_train_dataset = Dataset.from_generator(generate_sup_train_example)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# 訓練セットの形式と事例数を確認する
print(sup_train_dataset)

Dataset({
    features: ['premise', 'entailment_hypothesis', 'contradiction_hypothesis'],
    num_rows: 173438
})


In [ ]:
# 訓練セットの内容を確認する
pprint(sup_train_dataset[0])
pprint(sup_train_dataset[1])

{'contradiction_hypothesis': '男 が 台所 の テーブル で 本 を 読んで い ます 。',
 'entailment_hypothesis': 'ガレージ に 男 が い ます 。',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'contradiction_hypothesis': '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 'entailment_hypothesis': '人 は 椅子 に 座って い ます 。',
 'premise': 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。'}


#### collate 関数の準備

In [ ]:
def sup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """訓練セットのミニバッチを作成"""
    premises = []
    hypotheses = []

    for example in examples:
        premises.append(example["premise"])

        entailment_hypothesis = example["entailment_hypothesis"]
        contradiction_hypothesis = example["contradiction_hypothesis"]

        hypotheses.extend(
            [entailment_hypothesis, contradiction_hypothesis]
        )

    # ミニバッチに含まれる前提文と仮説文にトークナイザを適用する
    tokenized_premises = tokenizer(
        premises,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )
    tokenized_hypotheses = tokenizer(
        hypotheses,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 前提文と仮説文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（前提文）に対して
    # 2*i列目の要素（仮説文）が正例ペアとなる
    labels = torch.arange(0, 2 * len(premises), 2)

    return {
        "tokenized_texts_1": tokenized_premises,
        "tokenized_texts_2": tokenized_hypotheses,
        "labels": labels,
    }

#### モデルの準備

In [ ]:
# 教師ありSimCSEのモデルを初期化する
sup_model = SimCSEModel(base_model_name, mlp_only_train=False)

#### `Trainer` の準備

In [ ]:
# 教師ありSimCSEの訓練のハイパーパラメータを設定する
sup_training_args = TrainingArguments(
    output_dir="outputs_sup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=128,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=128,  # 評価時のバッチサイズ
    learning_rate=5e-5,  # 学習率
    num_train_epochs=3,  # 訓練エポック数
    eval_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
)

In [ ]:
# 教師ありSimCSEのTrainerを初期化する
sup_trainer = SimCSETrainer(
    model=sup_model,
    args=sup_training_args,
    data_collator=sup_train_collate_fn,
    train_dataset=sup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

#### 訓練の実行

In [ ]:
# 教師ありSimCSEの訓練を行う
sup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,1.450700,2.784451,0.790957
500,1.100500,2.694059,0.783635
750,1.030100,2.767380,0.785611
1000,0.969200,2.828739,0.789988
1250,0.929700,2.794001,0.794128
1500,0.832500,2.811610,0.792373
1750,0.762300,2.882742,0.800893
2000,0.754500,2.890922,0.799143
2250,0.748000,2.885197,0.785773
2500,0.730500,2.903657,0.795284


TrainOutput(global_step=4065, training_loss=0.8135068548561463, metrics={'train_runtime': 3013.1654, 'train_samples_per_second': 172.68, 'train_steps_per_second': 1.349, 'total_flos': 0.0, 'train_loss': 0.8135068548561463, 'epoch': 3.0})

#### 性能評価

In [ ]:
# 検証セットで教師ありSimCSEのモデルの評価を行う
sup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.882742166519165,
 'eval_spearman': 0.8008932898568148,
 'eval_runtime': 14.9486,
 'eval_samples_per_second': 832.92,
 'eval_steps_per_second': 6.556,
 'epoch': 3.0}

In [ ]:
# テストセットで教師ありSimCSEのモデルの評価を行う
sup_trainer.evaluate(test_dataset)

{'eval_loss': 2.598400354385376,
 'eval_spearman': 0.8190540442997506,
 'eval_runtime': 2.1308,
 'eval_samples_per_second': 683.795,
 'eval_steps_per_second': 5.632,
 'epoch': 3.0}

#### Google ドライブへの保存

In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!cp -r outputs_sup_simcse drive/MyDrive/llm-book